In [ ]:
import polars as pl
from expr_engine import ExprEngine

In [ ]:
expr = ExprEngine()
expr.init()

In [ ]:
df = pl.scan_parquet("../../../example_data/tools/crypto.parquet")

In [ ]:
df = df.with_columns(pl.col("open_time").alias("datetime"))

In [ ]:
df = df.sort(by=["symbol","datetime"])

In [ ]:
    f_return = "($close/Ref($close, 1)-1)"
    f_adv5 = "Mean($money, 5)"
    f_adv10 = "Mean($money, 10)"
    f_adv15 = "Mean($money, 15)"
    f_adv20 = "Mean($money, 20)"
    f_adv30 = "Mean($money, 30)"
    f_adv40 = "Mean($money, 40)"
    f_adv50 = "Mean($money, 50)"
    f_adv60 = "Mean($money, 60)"
    f_adv120 = "Mean($money, 120)"
    f_adv180 = "Mean($money, 180)"

In [ ]:
exp = expr.get_expression(f_return)


In [ ]:
 [[([(col("close")) / (col("close").shift().over([col("symbol")]))]) - (1)]] 

In [ ]:
df = df.filter(~pl.col("symbol").is_null()).collect()

In [ ]:
df = df.lazy()

In [ ]:
exp = expr.get_expression(f_return)

In [ ]:
exp2 = expr.get_expression(f_adv5)

In [ ]:
df = df.with_columns(money=pl.col("close")*pl.col("volume"))

In [ ]:
df.with_columns(f_adv5 = exp2.expr)

In [ ]:
df.with_columns(f_adv5 = exp2.expr).collect()

In [ ]:
df.select(pl.col("money").rolling_mean(20).over([pl.col("symbol")]).alias("f_adv5"))

In [ ]:
 pl.show_versions()

In [ ]:
alpha001 =  f"CSRank(IdxMax(Power(If({f_return}<0, Std({f_return}, 20), $close), 2), 5))-0.5"

In [ ]:
expr3 = expr.get_expression(alpha001)

In [ ]:
df.with_columns(alpha101 = expr3.expr)

In [ ]:
pl.arange(0,pl.count()).alias("index").groupby_rolling("index",period=5).agg(self._hs.product())

In [ ]:
expr3.batch_update(df)

In [ ]:
x = df.with_columns(pl.col("close").rolling_apply(lambda s: s.sum(),window_size = 5).over("symbol"))

In [ ]:
%%time
x.collect()

In [ ]:
%%time
#df.with_columns(pl.arange(0,pl.count()).alias("index")).groupby_rolling("index",period="5i").agg(pl.col("close").mean().over("symbol")).collect()

In [ ]:
df.select(pl.col("close").rolling_apply(lambda x:pl.rank(),5)).collect()

In [ ]:
import numpy as np

In [ ]:
%%time
df.select(pl.col("close").rolling_mean(5).over("symbol")).collect()

In [ ]:
df.columns

In [ ]:
%%time
df.select(pl.col("close").rolling_apply(lambda x:x.sum(),window_size=5).over("symbol")).collect()

In [ ]:
%%time
w = df.select(pl.col("close").rolling_apply(lambda x:x.sum(),window_size = 5).over("symbol")).collect()

In [ ]:
%%time
w = df.groupby_rolling("index",period="5i",by="symbol").agg(pl.col("close").sum())

In [ ]:
%%time
w.collect()

In [ ]:
pf = df.collect().